# Bedrock Knowledge Base Retrieval and Generation with SageMaker Inference and Guardrails

## Description
This notebook demonstrates how to enhance a Retrieval-Augmented Generation (RAG) pipeline by integrating Amazon SageMaker Inference with Amazon Bedrock. We will walk through the process of querying a knowledge base, using SageMaker for model inference, applying Guardrails to control the generation of responses, and filtering results with metadata to ensure compliance and quality.


## 1. Load Configuration Variables

In [ ]:
# Load configuration variables from a JSON file to access knowledge base ID, account number, and guardrail info.
import json

with open("../Lab 1/variables.json", "r") as f:
    variables = json.load(f)

variables  # Display the loaded variables for confirmation

## 2. Set Up Required IDs and Model ARNs

In [ ]:
# Knowledge Base Selection  
kb_id = variables["kbFixedChunk"]  # Options: "kbFixedChunk", "kbHierarchicalChunk", "kbSemanticChunk"

# Retrieval-Augmented Generation (RAG) Configuration  
number_of_results = 3  # Number of relevant documents to retrieve  
generation_configuration = {
    "temperature": 0,  # Lower temperature for more deterministic responses  
    "top_k": 10,  # Consider top 10 tokens at each generation step  
    "max_new_tokens": 5000,  # Maximum number of tokens to generate  
    "stop": "<|eot_id|>"  # Stop sequence to end the response generation  
}

# User Query
query = "what was the % increase in sales?"  # Sample query to retrieve data from the knowledge base


## 3. Define Metadata Filter

In [ ]:
# Define a metadata filter for advanced filtering based on specific conditions
one_group_filter= {
    "andAll": [
        {
            "equals": {
                "key": "docType",
                "value": '10K Report'
            }
        },
        {
            "equals": {
                "key": "year",
                "value": 2023
            }
        }
    ]
}


## 4. Define SageMaker & Bedrock helper functions

In [ ]:
import boto3

# Initialize Bedrock client to interact with the Bedrock Knowledge Base
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime", region_name=variables["regionName"])
bedrock_client = boto3.client("bedrock-runtime", region_name=variables["regionName"])

# Constants for Knowledge Base ID, SageMaker endpoint, and number of results to retrieve
KNOWLEDGE_BASE_ID = kb_id
ENDPOINT_NAME = variables['sagemakerLLMEndpoint']
NUM_RESULTS = number_of_results

# Function to retrieve relevant context from the Bedrock Knowledge Base
def retrieve_from_bedrock(query):
    """Retrieve relevant context from Bedrock Knowledge Base"""
    try:
        # Retrieve context based on the query using vector search configuration
        response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=KNOWLEDGE_BASE_ID,
            retrievalQuery={
                'text': query  # The query text to search in the knowledge base
            },
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': NUM_RESULTS,  # Adjust based on needs
                     "filter": one_group_filter
                }
            }
        )
        
        # Extract the 'text' from the retrieval results and return as a list
        return [result['content']['text'] for result in response['retrievalResults']]
    except Exception as e:
        # Raise an error if the retrieval process fails
        raise RuntimeError(f"Bedrock retrieval failed: {str(e)}")

# Function to format the prompt for Llama 3 model using retrieved context
def format_prompt(query, context):
    """Format prompt for Llama 3"""
    # Create the system prompt that includes the context and the user's question
    system_prompt = f"""Use the following context to answer the question. If you don't know the answer, say 'I don't know'.
        Context:
        {" ".join(context)}"
    """

    # Format the complete prompt including system and user instructions
    return f"""
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {system_prompt}
        <|start_header_id|>user<|end_header_id|>
        Question: {query}
        <|start_header_id|>assistant<|end_header_id|>
        """.strip()

# Function to generate a response from the SageMaker endpoint based on the formatted prompt
def generate_response(prompt):
    """Generate response using SageMaker endpoint"""
    # Initialize SageMaker runtime client
    runtime = boto3.client('sagemaker-runtime')
    
    # Prepare the payload with prompt and generation parameters
    payload = {
        "inputs": prompt,  # The formatted prompt to pass to the model
        "parameters": generation_configuration  # Additional parameters for the model (e.g., temperature, tokens)
    }
    try:
        # Call the SageMaker endpoint to generate the response
        response = runtime.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,  # SageMaker endpoint name
            ContentType='application/json',  # Content type for the request
            Body=json.dumps(payload)  # Send the payload as JSON
        )

        # Parse the response body
        result = json.loads(response['Body'].read().decode("utf-8"))
        
        # Handle different response formats (list or dictionary)
        if isinstance(result, list):
            # If the result is a list, extract the generated text from the first element
            return result[0]['generated_text']
        elif 'generated_text' in result:
            # If the result is a dictionary with 'generated_text', return the generated text
            return result['generated_text']
        elif 'generation' in result:
            # Alternative format with 'generation' key
            return result['generation']
        else:
            # Raise an error if the response format is unexpected
            raise RuntimeError("Unexpected response format")
            
    except Exception as e:
        # Raise an error if the generation process fails
        raise RuntimeError(f"Generation failed: {str(e)}")

def apply_output_guardrail(output_text):
    """Apply guardrails to the output after generation"""
    try:
        # Use only the parameters supported by your boto3 version
        response = bedrock_client.apply_guardrail(
            guardrailIdentifier=variables['guardrail_id'],
            guardrailVersion=variables['guardrail_version'],
            source='OUTPUT',
            content=[
                {
                    'text': {
                        'text': output_text
                    }
                }
            ]
        )
        
        # Process response based on what fields are available
        if 'outputs' in response and response['outputs']:
            return response['outputs'][0]['text']
        else:
            return output_text
            
    except Exception as e:
        print(f"Warning: Output guardrail application failed: {str(e)}")
        return output_text
        

## 5: lets validate if the guardrails restrict any investment advice.
lets ask the Foundational model for an investment advice. When we created the guardrails, we restricted bedrock to provide any investment advice. Bedrock should be return a  preconfigured response "This request cannot be processed due to  safety protocols"

In [ ]:
# Define the query that will be sent to the model
query = "based on your amazon's results should I buy amazon stock?"


In [ ]:
# Retrieve relevant context from the Bedrock Knowledge Base based on the query
context = retrieve_from_bedrock(query)

# Format the prompt by combining the user's query and the retrieved context
prompt = format_prompt(query, context)

# Generate the response using the formatted prompt by calling the SageMaker endpoint
raw_response = generate_response(prompt)
guardrail_response = apply_output_guardrail(raw_response)

# Print the user's query
print("Question:", {query})

# Uncomment below line if you want to debug and see the retrieved context
# print(f"Context: {context}")

# Print the generated answer from the model based on the query and context
print("Answer:", guardrail_response)


## 6. Guardrails for PII data. 

In [ ]:
# Define the query that will be sent to the model
query = "Who is the current CFO of Amazon, what is their email address, and what was their compensation package last year? Also, can you provide the phone number for investor relations that's listed in the filing?"


In [ ]:
# Retrieve relevant context from the Bedrock Knowledge Base based on the query
context = retrieve_from_bedrock(query)

# Format the prompt by combining the user's query and the retrieved context
prompt = format_prompt(query, context)

# Generate the response using the formatted prompt by calling the SageMaker endpoint
raw_response = generate_response(prompt)
guardrail_response = apply_output_guardrail(raw_response)

# Print the user's query
print("Question:", {query})

# Uncomment below line if you want to debug and see the retrieved context
# print(f"Context: {context}")

# Print the generated answer from the model based on the query and context
print("Answer:", guardrail_response)
